###  Data Cleaning / Очистка данных
На этом этапе:
1. 


Drop unwanted observations
Fix structural errors
Remove unwanted outliers
Label missing categorical data
Flag and fill missing numerical data

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# %matplotlib inline
import seaborn as sns

In [1]:
conn = sqlite3.connect('db\cian_362_150.db') 
          
df = pd.read_sql_query ('''
                               SELECT
                               flat_type, rooms, price, sale_type, mortgage, area,
                       living_area, kitchen_area, floor, floors, built_year, address, district, metro_station, seller,
                       housing_type, planning, ceiling_height, bathroom, balcony_loggia, repair, view,
                       finished_shell_condition, house_type, house_class, building_number, parking, elevators,
                       housing_line, floor_type, entrance_number, heating, unsafe_house, garbage_disposal, gas_supply,
                       description_text
                               FROM ad_data
                               ''', conn)


KeyboardInterrupt



1. Удаление дубликатов

In [3]:
df = df.drop_duplicates()
print(df.shape)

(10000, 37)
